In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
lipo_chemberta2_valid2 = pd.read_csv('./chemberta2/results/lipo/chemberta2_valid2_lipo_2_predictions.csv')
lipo_molformer_valid2 = pd.read_csv('./molformer/results/lipo/molformer_valid2_lipo_2.csv')
lipo_molbert_valid2 = pd.read_csv('./molbert/results/lipo/molbert_valid2_lipo_2.csv')

# Load the test data for each model
lipo_chemberta2_test = pd.read_csv('./chemberta2/results/lipo/chemberta2_test_lipo_2_predictions.csv')
lipo_molformer_test = pd.read_csv('./molformer/results/lipo/molformer_test_lipo_2.csv')
lipo_molbert_test = pd.read_csv('./molbert/results/lipo/molbert_test_lipo_2.csv')

train_mean = 2.1799722222222226
train_sd = 1.2045431891731355

# features

lipo_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/lipo/chemberta2_valid2_lipo_2_features.csv')
lipo_chemberta2_features_test = pd.read_csv('./chemberta2/features/lipo/chemberta2_test_lipo_2_features.csv')

lipo_molformer_features_valid2 = pd.read_csv('./molformer/features/lipo/molformer_valid2_lipo_2_features.csv')
lipo_molformer_features_test = pd.read_csv('./molformer/features/lipo/molformer_test_lipo_2_features.csv')

lipo_molbert_features_valid2 = pd.read_csv('./molbert/features/lipo/molbert_valid2_lipo_2_features.csv')
lipo_molbert_features_test = pd.read_csv('./molbert/features/lipo/molbert_test_lipo_2_features.csv')

For Lipo (Regression)

In [2]:
# Preparing the actual and predicted values
# Chemberta2
lipo_chemberta_actual = lipo_chemberta2_test['target'] 
lipo_chemberta_pred = lipo_chemberta2_test['pred_raw']

# Molformer
lipo_molformer_actual = lipo_molformer_test['target']
lipo_molformer_pred = lipo_molformer_test['pred_raw']

# molbert
lipo_molbert_actual = lipo_molbert_test['target_raw']
lipo_molbert_pred = lipo_molbert_test['pred_raw']

In [3]:
# Calculating metrics
lipo_metrics_results = {}

for model_name, actual, pred in [("Chemberta2", lipo_chemberta_actual, lipo_chemberta_pred),
                                 ("Molformer", lipo_molformer_actual, lipo_molformer_pred),
                                 ("Molbert", lipo_molbert_actual, lipo_molbert_pred)]:
    lipo_metrics_results[model_name] = {
        "MAE": mean_absolute_error(actual, pred),
        "RMSE": np.sqrt(mean_squared_error(actual, pred)),
        "R2 Score": r2_score(actual, pred),
        "Correlation": pearsonr(actual, pred)[0]  # Only record the correlation coefficient
    }

lipo_metrics_results

{'Chemberta2': {'MAE': 0.47080364152170984,
  'RMSE': 0.6194242164406615,
  'R2 Score': 0.725099924775132,
  'Correlation': 0.8597960575490352},
 'Molformer': {'MAE': 0.45518342900000003,
  'RMSE': 0.6064332745501879,
  'R2 Score': 0.7365097534650544,
  'Correlation': 0.867404233883593},
 'Molbert': {'MAE': 0.5100067769880953,
  'RMSE': 0.6507887204149233,
  'R2 Score': 0.6965560154119396,
  'Correlation': 0.8370118646634633}}

In [4]:
# standardized valid2 labels
lipo_y_ensemble_valid2 = (lipo_chemberta2_valid2['target'] - train_mean)/train_sd

# Create the features for the ensemble from the prediction probabilities of being in class 1
lipo_X_ensemble_valid2 = pd.concat([
    lipo_chemberta2_valid2['pred_z'] - lipo_y_ensemble_valid2,
    lipo_molformer_valid2['pred_z'] - lipo_y_ensemble_valid2, 
    lipo_molbert_valid2['pred_z'] - lipo_y_ensemble_valid2,
    # add features from training set
    lipo_chemberta2_valid2['pred_z'],
    lipo_molformer_valid2['pred_z'],
    lipo_molbert_valid2['pred_z']
], axis=1)

# change feature names of the ensemble so that they are unique
lipo_X_ensemble_valid2.columns = ['residuals_chemberta', 'residuals_molformer', 'residuals_molbert', 'chemberta', 'molformer', 'molbert']

In [5]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss
import numpy as np

# Set seeds for reproducibility
np.random.seed(0)

# Compute residuals
chemberta_y_residual = lipo_chemberta2_valid2['pred_z'] - lipo_y_ensemble_valid2
molformer_y_residual = lipo_molformer_valid2['pred_z'] - lipo_y_ensemble_valid2
molbert_y_residual = lipo_molbert_valid2['pred_z'] - lipo_y_ensemble_valid2

# For input features, we use pred_z and the other features
chemberta_X = pd.concat([lipo_chemberta2_valid2['pred_z'], 
                         lipo_chemberta2_features_valid2.iloc[:, 2:]], axis=1)
molformer_X = pd.concat([lipo_molformer_valid2['pred_z'], 
                         lipo_molformer_features_valid2.iloc[:, 1:]], axis=1)
molbert_X = pd.concat([lipo_molbert_valid2['pred_z'], 
                       lipo_molbert_features_valid2.iloc[:, 1:]], axis=1)

# Standardize each dataset
scaler_chemberta = StandardScaler().fit(chemberta_X)
scaler_molformer = StandardScaler().fit(molformer_X)
scaler_molbert = StandardScaler().fit(molbert_X)

chemberta_X_scaled = scaler_chemberta.transform(chemberta_X)
molformer_X_scaled = scaler_molformer.transform(molformer_X)
molbert_X_scaled = scaler_molbert.transform(molbert_X)

# Define RMSE scorer
def rmse_scorer(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Define the hyperparameter space for regression using continuous distributions
xgb_hyperopt_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 50),
    'max_depth': hp.quniform('max_depth', 3, 7, 2),
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

# Objective function for hyperopt with XGBRegressor
def objective(params, X_train, y_train):
    # Convert float outputs of hp.quniform to int for certain parameters
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    
    # Initialize XGBRegressor with the given parameters
    model = xgb.XGBRegressor(**params, random_state=0)
    
    # RMSE as the scoring metric
    rmse = make_scorer(rmse_scorer, greater_is_better=False)
    score = cross_val_score(model, X_train, y_train, scoring=rmse, cv=5)
    
    # Minimize the negative RMSE score
    return {'loss': -score.mean(), 'status': STATUS_OK}

# Function to optimize and train the model for each dataset
def optimize_and_train(X_train, y_train):
    # Run the Bayesian optimization
    trials = Trials()
    best_params = fmin(fn=lambda params: objective(params, X_train, y_train), 
                       space=xgb_hyperopt_space, 
                       algo=tpe.suggest, 
                       max_evals=50, 
                       trials=trials,
                       rstate=np.random.default_rng(0),  # Seed for reproducibility in hyperopt
                       early_stop_fn=no_progress_loss(10))

    # Convert the best hyperparameters to the correct data types
    best_params['n_estimators'] = int(best_params['n_estimators'])
    best_params['max_depth'] = int(best_params['max_depth'])

    # Initialize and train the XGBoost model with the best parameters
    model = xgb.XGBRegressor(**best_params, random_state=0)
    model.fit(X_train, y_train)
    
    return model

# Optimizing and training for chemberta dataset
print("Optimizing for chemberta dataset...")
best_model_chemberta = optimize_and_train(chemberta_X_scaled, chemberta_y_residual)

# Optimizing and training for molformer dataset
print("Optimizing for molformer dataset...")
best_model_molformer = optimize_and_train(molformer_X_scaled, molformer_y_residual)

# Optimizing and training for molbert dataset
print("Optimizing for molbert dataset...")
best_model_molbert = optimize_and_train(molbert_X_scaled, molbert_y_residual)

# Print a confirmation message
print("Models trained for chemberta, molformer, and molbert datasets.")

Optimizing for chemberta dataset...
 64%|██████▍   | 32/50 [04:18<02:25,  8.07s/trial, best loss: 0.51995281374393]  
Optimizing for molformer dataset...
 78%|███████▊  | 39/50 [06:46<01:54, 10.43s/trial, best loss: 0.5174326563201829]
Optimizing for molbert dataset...
 46%|████▌     | 23/50 [03:51<04:31, 10.04s/trial, best loss: 0.5409159849458041]
Models trained for chemberta, molformer, and molbert datasets.


In [6]:
# ChemBERTa
chemberta_X_test = pd.concat([lipo_chemberta2_test['pred_z'], 
                              lipo_chemberta2_features_test.iloc[:, 2:]], axis=1)

# MolFormer
molformer_X_test = pd.concat([lipo_molformer_test['pred_z'], 
                              lipo_molformer_features_test.iloc[:, 1:]], axis=1)

# MolBERT
molbert_X_test = pd.concat([lipo_molbert_test['pred_z'], 
                            lipo_molbert_features_test.iloc[:, 1:]], axis=1)

# Scale the test data using the corresponding scalers
chemberta_X_test_scaled = scaler_chemberta.transform(chemberta_X_test)
molformer_X_test_scaled = scaler_molformer.transform(molformer_X_test)
molbert_X_test_scaled = scaler_molbert.transform(molbert_X_test)

# Predict residuals for the test set using the trained GroupLasso models
y_pred_residuals_chemberta_test = best_model_chemberta.predict(chemberta_X_test_scaled)
y_pred_residuals_molformer_test = best_model_molformer.predict(molformer_X_test_scaled)
y_pred_residuals_molbert_test = best_model_molbert.predict(molbert_X_test_scaled)

# Create a DataFrame to store the predicted residuals for the test set
predicted_residuals_test = pd.DataFrame({
    'Residuals_Chemberta': y_pred_residuals_chemberta_test,
    'Residuals_Molformer': y_pred_residuals_molformer_test,
    'Residuals_Molbert': y_pred_residuals_molbert_test
})

# Load test data into ensemble DataFrame, adding the predicted residuals and original predictions
lipo_X_ensemble_test = pd.concat([
    predicted_residuals_test,
    lipo_chemberta2_test['pred_z'],
    lipo_molformer_test['pred_z'],  
    lipo_molbert_test['pred_z']
], axis=1)

# Rename feature columns so that they are unique
lipo_X_ensemble_test.columns = ['residuals_chemberta', 'residuals_molformer', 'residuals_molbert', 'chemberta', 'molformer', 'molbert']

# True test labels
lipo_y_ensemble_test = lipo_chemberta2_test['target']

In [7]:
# scale the features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
lipo_X_ensemble_valid2_scaled = scaler.fit_transform(lipo_X_ensemble_valid2)
lipo_X_ensemble_test_scaled = scaler.transform(lipo_X_ensemble_test)

lipo_X_ensemble_valid2_scaled = pd.DataFrame(lipo_X_ensemble_valid2_scaled, columns=lipo_X_ensemble_valid2.columns)
lipo_X_ensemble_test_scaled = pd.DataFrame(lipo_X_ensemble_test_scaled, columns=lipo_X_ensemble_test.columns)

In [8]:
lipo_X_ensemble_valid2_selected = lipo_X_ensemble_valid2_scaled
lipo_X_ensemble_test_selected = lipo_X_ensemble_test_scaled

# check shapes
print(lipo_X_ensemble_valid2_selected.shape)
print(lipo_X_ensemble_test_selected.shape)

(840, 6)
(420, 6)


In [9]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss
import random

# Set seeds for reproducibility
np.random.seed(0)
random.seed(0)

# Define the hyperparameter space using continuous distributions
lipo_xgb_hyperopt_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 50),
    'max_depth': hp.quniform('max_depth', 3, 7, 2),
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

# Correctly define the RMSE scorer function
def rmse_scorer(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Convert float outputs of hp.quniform to int for certain parameters
def objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    model = xgb.XGBRegressor(**params, random_state=0)
    
    # Cross-validated RMSE as the objective
    score = cross_val_score(model, lipo_X_ensemble_valid2_selected, lipo_y_ensemble_valid2, 
                            scoring=make_scorer(rmse_scorer, greater_is_better=False), cv=5)
    
    # Minimize the positive RMSE (already negative from scoring)
    return {'loss': -score.mean(), 'status': STATUS_OK}

# Run the Bayesian optimization
trials = Trials()
lipo_xgb_best_params = fmin(fn=objective, 
                            space=lipo_xgb_hyperopt_space, 
                            algo=tpe.suggest, 
                            max_evals=50, 
                            trials=trials,
                            rstate=np.random.default_rng(0),  # Seed for hyperopt
                            early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", lipo_xgb_best_params)


 50%|█████     | 25/50 [00:09<00:09,  2.74trial/s, best loss: 0.09354631611815283]
Best hyperparameters: {'colsample_bytree': 0.9558186054871323, 'learning_rate': 0.09634469734464615, 'max_depth': 4.0, 'n_estimators': 200.0, 'subsample': 0.8082925817075997}


In [10]:
# fit the model with the best hyperparameters
# Convert parameters obtained from Hyperopt to the correct data type
lipo_xgb_best_params['n_estimators'] = int(lipo_xgb_best_params['n_estimators'])
lipo_xgb_best_params['max_depth'] = int(lipo_xgb_best_params['max_depth'])

# Initialize and train the XGBoost model with the best parameters
lipo_xgb_model = xgb.XGBRegressor(**lipo_xgb_best_params, random_state=0)
lipo_xgb_model.fit(lipo_X_ensemble_valid2_selected, lipo_y_ensemble_valid2)

# Predict the test set
lipo_xgb_best_pred = lipo_xgb_model.predict(lipo_X_ensemble_test_selected) * train_sd + train_mean

# Calculate the metrics
lipo_xgb_best_metrics = {
    "MAE": mean_absolute_error(lipo_y_ensemble_test, lipo_xgb_best_pred),
    "RMSE": np.sqrt(mean_squared_error(lipo_y_ensemble_test, lipo_xgb_best_pred)),
    "R2 Score": r2_score(lipo_y_ensemble_test, lipo_xgb_best_pred),
    "Correlation": pearsonr(lipo_y_ensemble_test, lipo_xgb_best_pred)[0]  # Only record the correlation coefficient
}

lipo_xgb_best_metrics

{'MAE': 0.4122656939824422,
 'RMSE': 0.5339607690606882,
 'R2 Score': 0.7957240691290287,
 'Correlation': 0.8923536936803771}

In [11]:
# create a table to record all metrics for lipo
lipo_metrics_results["XGBoost"] = lipo_xgb_best_metrics

lipo_metrics_df = pd.DataFrame(lipo_metrics_results).T
# keep 3 digits after the decimal point
lipo_metrics_df = lipo_metrics_df.round(3)

# export table to csv
lipo_metrics_df.to_csv('./split2_lipo_metrics_xgb.csv')